In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
files_path = "../../data/anime_codes_by_genre/"
anime_files = os.listdir(files_path)

In [3]:
anime_genres = [file.lower().split(".")[0] for file in anime_files]

In [4]:
one_hot = OneHotEncoder(handle_unknown="ignore")

In [5]:
one_hot.fit([[genre] for genre in anime_genres])

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='ignore', sparse=True)

In [7]:
animes_with_genres = dict()
for file in anime_files:
    current_df = pd.read_csv(os.path.join(files_path, file), encoding="utf-8")
    current_df = current_df[current_df.rating >= 6.5]
    for code in current_df.code.to_list():
        animes_with_genres.setdefault(code, np.zeros((1,len(anime_files))))
        genre = file.lower().split(".")[0]
        animes_with_genres[code] += one_hot.transform([[genre]]).toarray()

In [8]:
pickle.dump(animes_with_genres, open("./anime_genres.pickle", "wb"))

In [ ]:
def get_recommendations(anime_code: int):
    anime_genres = animes_with_genres.get(anime_code)[0]
    indexes = [i for i in range(0, len(anime_genres)) if anime_genres[i] == 1]
    top_recommendations = []
    for key, values in animes_with_genres.items():
        similarity = sum(values[0][indexes])
        if key!=anime_code:
            if not top_recommendations or len(top_recommendations)<5:
                top_recommendations.append([key, similarity])
                continue            
            for idx, recommendation in enumerate(top_recommendations):
                if similarity > recommendation[1]:
                    top_recommendations[idx] = [key, similarity]
                    break
                    
    return top_recommendations

In [ ]:
len(animes_with_genres)